In [3]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=61e905133522b4fbe0b4090554260be7bf08d8960561c99770a94463ed5819c6
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [4]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import pickle

sns.set_theme()
tqdm.pandas()

# Modular function for the end-to-end combined pipeline:

In [9]:
MODEL_PATH = "../../models/combined_approach/kmeans.pkl"
THRESHOLD = 0.95
kmeans = pickle.load(open(MODEL_PATH, 'rb'))
mpnet = SentenceTransformer("all-mpnet-base-v2")

def compute_distance_of_closest_cluster(df):
    cluster_distances = kmeans.transform(mpnet.encode(df["query"].values))
    cluster_min_distances = cluster_distances.min(axis=1)
    df["min_distances"] = cluster_min_distances
    
def predict_oos(df):
  '''
  input df is a Pandas dataframe with the text input under the column named 'query'
  ''' 
  sentence_embeddings = []
  embedding_matrix = mpnet.encode(df["query"].values)
  for i in range(embedding_matrix.shape[0]):
    sentence_embeddings.append(embedding_matrix[i])  

  df["mpnet_embeddings_sentence"] = sentence_embeddings
  compute_distance_of_closest_cluster(df)
  df["pred"] = df.min_distances.apply(lambda x: "oos" if x>THRESHOLD else "in-scope")
  return df

In [10]:
df_test = pd.read_csv("../../data/original_dataset/original_test_data.csv")

In [11]:
df_test = predict_oos(df_test)

In [12]:
df_test["binary_label"] =  df_test.label.apply(lambda x: "oos" if x=="oos" else "in-scope")
pd.DataFrame(classification_report(df_test["binary_label"], df_test["pred"], output_dict=True)).T

,precision,recall,f1-score,support
in-scope,0.950548,0.982444,0.966233,4500.000000
oos,0.906949,0.770000,0.832883,1000.000000
accuracy,0.943818,0.943818,0.943818,0.943818
macro avg,0.928749,0.876222,0.899558,5500.000000
weighted avg,0.942621,0.943818,0.941988,5500.000000
